In [1]:
# !pip install pyarrow
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

import pandas as pd

def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)
        
        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
        
        num_col = df.columns
        print(f'The below columns are in the data for January {df.columns}, however the total number of columns is {len(df.columns)}')
        
        df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime'] 
        df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)
        print(f'Standard Deviation for duration is:', df['duration'].describe())
        
        df_records = df.shape[0]
        df_outliers_dropped = df[(df.duration >= 1) & (df.duration <= 60)]
        df_outliers_records = df_outliers_dropped.shape[0]
        frac_records = (df_outliers_records / df_records) * 100
        print(f'Fraction of records left =', frac_records)
        
    
    return df

In [2]:
df_jan = read_dataframe('./yellow_tripdata_2022-01.parquet')


The below columns are in the data for January Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object'), however the total number of columns is 19
Standard Deviation for duration is: count    2.463931e+06
mean     1.421220e+01
std      4.644531e+01
min     -3.442400e+03
25%      6.316667e+00
50%      1.018333e+01
75%      1.616667e+01
max      8.513183e+03
Name: duration, dtype: float64
Fraction of records left = 98.27547930522405


In [3]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,17.816667
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,8.400000
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,8.966667
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,10.033333
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,37.533333


In [4]:
df_feb = read_dataframe('./yellow_tripdata_2022-02.parquet')

The below columns are in the data for January Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object'), however the total number of columns is 19
Standard Deviation for duration is: count    2.979431e+06
mean     1.565368e+01
std      4.726394e+01
min     -9.833333e-01
25%      6.950000e+00
50%      1.125000e+01
75%      1.783333e+01
max      5.489383e+03
Name: duration, dtype: float64
Fraction of records left = 97.9444397269143


In [5]:
df_feb.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.40,1.0,N,138,252,1,17.0,1.75,0.5,3.90,0.00,0.3,23.45,0.0,1.25,12.433333
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.40,1.0,N,138,41,2,21.0,1.75,0.5,0.00,6.55,0.3,30.10,0.0,1.25,17.550000
2,1,2022-02-01 00:03:20,2022-02-01 00:26:59,1.0,12.50,1.0,N,138,200,2,35.5,1.75,0.5,0.00,6.55,0.3,44.60,0.0,1.25,23.650000
3,2,2022-02-01 00:08:00,2022-02-01 00:28:05,1.0,9.88,1.0,N,239,200,2,28.0,0.50,0.5,0.00,3.00,0.3,34.80,2.5,0.00,20.083333
4,2,2022-02-01 00:06:48,2022-02-01 00:33:07,1.0,12.16,1.0,N,138,125,1,35.5,0.50,0.5,8.11,0.00,0.3,48.66,2.5,1.25,26.316667


In [6]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_jan_new = df_jan.copy()
df_feb_new = df_feb.copy()

df_jan_new[categorical] = df_jan_new[categorical].astype(str)
df_feb_new[categorical] = df_feb_new[categorical].astype(str)

jan_train_df_dicts = df_jan_new[categorical + numerical].to_dict(orient='records')
feb_train_df_dicts = df_feb_new[categorical + numerical].to_dict(orient='records')

d_vect = DictVectorizer()
X_train = d_vect.fit_transform(jan_train_df_dicts)
X_val = d_vect.transform(feb_train_df_dicts)

print(f'The dimensionality after the OHE is perfomed is:', X_train.shape[1])
# print(f'The dimensionality after the OHE is perfomed is:', X_val.shape[1])

The dimensionality after the OHE is perfomed is: 519


In [7]:
target = 'duration'
y_train = df_jan_new[target].values
y_val = df_feb_new[target].values

lr = LinearRegression()
lr_model = lr.fit(X_train, y_train)

y_train_pred = lr_model.predict(X_train)
y_val_pred = lr_model.predict(X_val)

rmse_val = mean_squared_error(y_val, y_val_pred, squared=False)
rmse_train = mean_squared_error(y_train, y_train_pred, squared=False)

print(f'The RMSE for the train set is:', rmse_train)
print(f'The RMSE for the train set is:', rmse_val)

The RMSE for the train set is: 46.01451722545104
The RMSE for the train set is: 46.792389952273325
